# Tạo **Adaptation Vector** theo hướng **PT aligned → FT normalized**

- **Mục đích**
  - Với mỗi câu và mỗi lớp (layer), tính vector thích nghi (Adaptation Vector – *AV*) để mô tả sai khác giữa
    **vector FT đã chuẩn hoá** và **vector PT đã xoay về không gian FT**.
  - Dùng cho các bước phân tích/huấn luyện tiếp theo (ví dụ làm đặc trưng bổ sung).

- **Đầu vào**
  - Thư mục **FT (normalized)**:
    - Train GramVar: `.../finetuned_train_vectors_normalized_gramvar/`
    - Train ParaVE: `.../finetuned_train_vectors_normalized_parave/`
    - Test  GramVar: `.../finetuned_test_vectors_normalized_gramvar/`
    - Test  ParaVE: `.../finetuned_test_vectors_normalized_parave/`
  - Thư mục **PT (aligned to FT)**:
    - Train GramVar: `.../pretrained_train_vectors_aligned_to_ft_gramvar/`
    - Train ParaVE: `.../pretrained_train_vectors_aligned_to_ft_parave/`
    - Test  GramVar: `.../pretrained_test_vectors_aligned_to_ft_gramvar/`
    - Test  ParaVE: `.../pretrained_test_vectors_aligned_to_ft_parave/`

> Mỗi file `.pt` là một **dict** có các khoá `layer_0` … `layer_12` (13 lớp), mỗi khoá là tensor `[num_tokens, 768]`.


- **Đầu ra**
  - Thư mục **Adaptation Vectors** mới (giữ nguyên cấu trúc con theo động từ):
    - Train GramVar: `.../adaptation_vectors_train_gramvar_pt_aligned/`
    - Train ParaVE: `.../adaptation_vectors_train_parave_pt_aligned/`
    - Test  GramVar: `.../adaptation_vectors_test_gramvar_pt_aligned/`
    - Test  ParaVE: `.../adaptation_vectors_test_parave_pt_aligned/`
  - Mỗi file `.pt` (một câu) là **dict**: `{ 'layer_k': Tensor[num_tokens, 768], ... }`.

- **Công thức**
  - Với mỗi lớp $k$:
  $$
  X^{\text{AV}}_k \;=\; X^{\text{FT(norm)}}_k \;-\; X^{\text{PT(aligned)}}_k
  $$
  - Chỉ thực hiện khi 2 tensor cùng **shape**; nếu lệch kích thước → **bỏ qua lớp đó**.

- **Quy trình**
  1. Duyệt đệ quy tất cả file `.pt` trong thư mục **FT (normalized)**.
  2. Suy ra file PT tương ứng bằng **relative path** trong thư mục **PT (aligned)**.
  3. Đọc 2 dict (`layer_k: X_k`), với từng lớp $k$ tính:
  $$
  X^{\text{AV}}_k = X^{\text{FT(norm)}}_k - X^{\text{PT(aligned)}}_k
  $$
  4. Ghi file **AV** sang thư mục output, giữ **relative path**.
  5. Hiển thị tiến trình bằng `tqdm`.

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import glob
from tqdm.auto import tqdm

# THIẾT LẬP CÁC ĐƯỜNG DẪN (THEO HƯỚNG PT ALIGNED)

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# --- DANH SÁCH CÁC BỘ DỮ LIỆU CẦN TÍNH ADAPTATION VECTOR ---
# Mỗi mục là một tuple chứa: (Tên, Thư mục input FT (chỉ normalized), Thư mục input PT (đã aligned), Thư mục output)
datasets_to_process = [
    # --- Dữ liệu Train ---
    (
        "Train_GramVar",
        os.path.join(drive_base_path, 'Vector Normalization/Train/Finetuned/finetuned_train_vectors_normalized_gramvar'),      # Input 1: FT đã chuẩn hóa
        os.path.join(drive_base_path, 'Vector Alignment/Train/pretrained_train_vectors_aligned_to_ft_gramvar'), # Input 2: PT đã xoay
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_train_gramvar_pt_aligned')      # Output mới
    ),
    (
        "Train_ParaVE",
        os.path.join(drive_base_path, 'Vector Normalization/Train/Finetuned/finetuned_train_vectors_normalized_parave'),
        os.path.join(drive_base_path, 'Vector Alignment/Train/pretrained_train_vectors_aligned_to_ft_parave'),
        os.path.join(drive_base_path, 'Adaptation Vector/Train/adaptation_vectors_train_parave_pt_aligned')
    ),
    # --- Dữ liệu Test ---
    (
        "Test_GramVar",
        os.path.join(drive_base_path, 'Vector Normalization/Test/Finetuned/finetuned_test_vectors_normalized_gramvar'),
        os.path.join(drive_base_path, 'Vector Alignment/Test/pretrained_test_vectors_aligned_to_ft_gramvar'),
        os.path.join(drive_base_path, 'Adaptation Vector/Test/adaptation_vectors_test_gramvar_pt_aligned')
    ),
    (
        "Test_ParaVE",
        os.path.join(drive_base_path, 'Vector Normalization/Test/Finetuned/finetuned_test_vectors_normalized_parave'),
        os.path.join(drive_base_path, 'Vector Alignment/Test/pretrained_test_vectors_aligned_to_ft_parave'),
        os.path.join(drive_base_path, 'Adaptation Vector/Test/adaptation_vectors_test_parave_pt_aligned')
    )
]

# HÀM TÍNH TOÁN ADAPTATION VECTOR

def calculate_adaptation_vectors(normalized_ft_data, aligned_pt_data):
    """
    Tính toán adaptation vector bằng cách lấy hiệu của vector FT (normalized) và PT (đã aligned).
    """
    adaptation_vectors = {}
    for layer_name in normalized_ft_data.keys():
        if layer_name in aligned_pt_data:
            ft_tensor = normalized_ft_data[layer_name]
            pt_tensor = aligned_pt_data[layer_name]

            if ft_tensor.shape == pt_tensor.shape:
                # Công thức: Adaptation Vector = Normalized FT - Aligned PT
                adaptation_vectors[layer_name] = ft_tensor - pt_tensor
            else:
                print(f"  CẢNH BÁO: Kích thước không khớp cho {layer_name}. Bỏ qua.")
    return adaptation_vectors

# THỰC THI CHƯƠNG TRÌNH

if __name__ == "__main__":
    print("--- Bắt đầu quá trình tạo Adaptation Vector (theo hướng PT Aligned) ---")

    for name, ft_input_dir, pt_input_dir, output_dir in datasets_to_process:
        print(f"\n=================================================")
        print(f"BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: {name}")
        print(f"=================================================")

        os.makedirs(output_dir, exist_ok=True)

        # Tìm tất cả các file vector FT (normalized) để làm file tham chiếu
        ft_vector_files = glob.glob(os.path.join(ft_input_dir, '**', '*.pt'), recursive=True)
        total_files = len(ft_vector_files)

        if total_files == 0:
            print(f"CẢNH BÁO: Không tìm thấy file vector nào trong '{ft_input_dir}'. Bỏ qua.")
            continue

        print(f"Tìm thấy {total_files} file vector để xử lý...")

        pbar = tqdm(total=total_files, desc=f"Tạo AV cho {name}", unit="file")

        for ft_path in ft_vector_files:
            try:
                # Suy ra đường dẫn file PT tương ứng
                relative_path = os.path.relpath(ft_path, ft_input_dir)
                pt_path = os.path.join(pt_input_dir, relative_path)

                # Tải dữ liệu của cả hai
                normalized_ft_data = torch.load(ft_path)
                aligned_pt_data = torch.load(pt_path)

                # Tính toán adaptation vector
                adaptation_vector_data = calculate_adaptation_vectors(normalized_ft_data, aligned_pt_data)

                # Tạo đường dẫn lưu file mới
                output_path = os.path.join(output_dir, relative_path)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                # Lưu file adaptation vector
                torch.save(adaptation_vector_data, output_path)

            except FileNotFoundError:
                pbar.write(f"LỖI: Không tìm thấy một trong các file sau. Bỏ qua.")
                pbar.write(f"  - FT Path: {ft_path}")
                pbar.write(f"  - PT Path: {pt_path}")
            except Exception as e:
                pbar.write(f"LỖI khi xử lý file {ft_path}: {e}")

            pbar.update(1)

        pbar.close()
        print(f"  -> Đã xử lý xong toàn bộ {total_files} file cho bộ dữ liệu '{name}'.")

    print("\n--- QUÁ TRÌNH TẠO ADAPTATION VECTOR ĐÃ HOÀN TẤT! ---")


- **Cấu trúc thư mục (minh hoạ)**
  ```bash
  .../finetuned_train_vectors_normalized_gramvar/
  ├── bind/
  │   ├── sentence_0.pt
  │   └── ...
  .../pretrained_train_vectors_aligned_to_ft_gramvar/
  ├── bind/
  │   ├── sentence_0.pt
  │   └── ...
  .../adaptation_vectors_train_gramvar_pt_aligned/
  ├── bind/
  │   ├── sentence_0.pt   # dict: layer_0..layer_12 (AV)
  │   └── ...